## Reference states

In this lesson, we will explore how we can initialize our system with a [*reference state*](gloss:reference) to help our variational algorithm converge faster. First, we will learn how to construct a reference state manually, and then explore several standard options that can be used in a variational algorithm.

![Reference Workflow](images/reference_workflow.png)

## Default state

A *reference state* $|\rho\rangle$ refers to the initial fixed start for our problem. To prepare a reference state, we need to apply the appropriate unitary $U_R$ at the start of our quantum circuit, such that $|\rho\rangle = U_R |0\rangle$. If you have an educated guess or a datapoint from an existing optimal solution, the variational algorithm will likely converge faster if you start with that *initial state*.

The simplest possible reference state is the default state, where we use the starting state of an $n$-qubit quantum circuit: $|0\rangle^{\otimes n}$. For the default state, our unitary operator $U_R \equiv I$. Due to its simplicity, the default state is a valid reference state used in many scenarios.

## Classical reference state

Suppose you have a $3$-qubit system and you want to start in the state $|001\rangle$ instead of the default state $|000\rangle$. This is an example of a purely classical reference state, and to construct it, you simply need to apply an [X gate](https://qiskit.org/documentation/stubs/qiskit.circuit.library.XGate.html) to qubit $0$ (following Qiskit's qubit ordering), as $|001\rangle = X_0 |000\rangle$.

In this case, our unitary operator is $U_R \equiv X_0$, which leads to the reference state $|\rho\rangle \equiv |001\rangle$.

In [ ]:
from qiskit import QuantumCircuit

qc = QuantumCircuit(3)
qc.x(0)

qc.draw("mpl")

## Quantum reference state

Suppose you aim to start with a more complex state that involves superposition and/or entanglement, such as $\frac{1}{\sqrt{2}}(|100\rangle+|111\rangle)$.

To obtain this state from $|000\rangle$, one approach is to use a [Hadamard gate](https://qiskit.org/documentation/stubs/qiskit.circuit.library.HGate.html) on qubit $0$ ($H_0$), a [CNOT (CX)](https://qiskit.org/documentation/stubs/qiskit.circuit.library.CXGate.html) gate with qubit $0$ as the control qubit and qubit $1$ as the target qubit ($CNOT_{01}$), and finally an $X$ gate applied to qubit $2$ ($X_2$).

In this scenario, our unitary operator is $U_{R} \equiv X_2CNOT_{01}H_0|000\rangle$, and our reference state is $|\rho\rangle \equiv \frac{1}{\sqrt{2}}(|100\rangle+|111\rangle)$.

In [ ]:
qc = QuantumCircuit(3)
qc.h(0)
qc.cx(0,1)
qc.x(2)

qc.draw("mpl")

## Constructing Reference States using template circuits

We can also use various template circuits, such as [`TwoLocal`](https://qiskit.org/documentation/stubs/qiskit.circuit.library.TwoLocal.html) which allows for expressing multiple tunable parameters and entanglements with ease. We will cover these template circuits in more detail in the next lesson, but we can use them for our reference states *if* we bind the parameters.:

In [ ]:
from qiskit.circuit.library import TwoLocal
from math import pi

reference_circuit = TwoLocal(2, "rx", "cz", entanglement="linear", reps=1)
theta_list = [pi/2, pi/3, pi/3, pi/2]

reference_circuit = reference_circuit.bind_parameters(theta_list)

reference_circuit.decompose().draw('mpl')

## Application-specific reference states

### Quantum Chemistry

In Quantum Chemistry, the *Hartree-Fock* state is an approximation of the ground state of an atom or molecule. If our goal is to create a variational algorithm to find the exact ground state, we can use this known classical approximation as a reference state to help our algorithm converge faster. In this example, we generate an Electronic Structure Problem for $H_{2}$ and use the problem's `num_spatial_orbitals` and `num_particles` to create our initial state.

In [ ]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.circuit.library import HartreeFock
from qiskit_nature.second_q.mappers import JordanWignerMapper, QubitConverter

driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.735",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

h2_problem = driver.run()

converter = QubitConverter(JordanWignerMapper())

h2_reference_state = HartreeFock(
    h2_problem.num_spatial_orbitals,
    h2_problem.num_particles,
    converter
)

h2_reference_state.decompose().draw('mpl')

### Quantum Machine Learning

In the context of a [variational quantum classifier (VQC)](https://learn.qiskit.org/course/machine-learning/variational-classification), the training data is encoded into a quantum state with a parameterized circuit known as a *feature map*, where each parameter value represents a data point from the training dataset. The [ZZFeatureMap](https://qiskit.org/documentation/stubs/qiskit.circuit.library.ZZFeatureMap.html) is a type of parameterized circuit that can be utilized to pass our data points ($x$) to this feature map.

In [ ]:
from qiskit.circuit.library import ZZFeatureMap

data = [0.1, 0.2]

zz_feature_map_reference = ZZFeatureMap(feature_dimension=2, reps=2)
zz_feature_map_reference = zz_feature_map_reference.bind_parameters(data)
zz_feature_map_reference.decompose().draw('mpl')

With this lesson, you learned how to initialize your system using:

- Default reference state
- Classical reference states
- Quantum reference states
- Application-specific reference states

Our high-level variational workload looks as follows:

![Reference Circuit](images/reference_circuit.png)

While reference states are fixed, initial starting points, we can use a *variational form* to define an *ansatz* to represent a collection of parametrized states for our variational algorithm to explore.